In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
!pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 3.8 MB/s eta 0:00:00
     -------------------------------------- 896.6/896.6 KB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 5.5 MB/s eta 0:00:00
     ------------------------------------ 439.2/439.2 KB 670.3 kB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 KB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 4.1 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 KB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 93

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import random
import tensorflow as tf

def seed_everything(seed:int=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

ModuleNotFoundError: No module named 'tensorflow'

# 데이터 로드

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
!unzip -o ml-100k.zip

'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## user

In [5]:
user_column_names =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user_df = pd.read_csv("ml-100k/u.user", sep='|', names=user_column_names)

## item

In [ ]:
item_column_names = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

item_df = pd.read_csv('ml-100k/u.item', sep='|', names=item_column_names, encoding='latin-1')

# 모델 구조
reference : 

https://github.com/LeeHyeJin91/Neural_CF

https://github.com/hexiangnan/neural_collaborative_filtering


## MLP

In [6]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [6]:
class MLP():
    # 사용할 layer 작성
    def __init__(self,user_num,item_num, layers):
        
        # user embedding
        user = Input(shape=(1,))
        # user_embedding = Embedding(user_num, 32, input_length=user.shape[1])(user)
        user_embedding = Embedding(user_num, 32, input_length=user.shape[1])(user)
        user_embedding = Flatten()(user_embedding)
        
        # item embedding
        item = Input(shape=(1,))
        # item_embedding = Embedding(item_num, 32, input_length = item.shape[1])(item)
        item_embedding = Embedding(item_num, 32, input_length = item.shape[1])(item)
        item_embedding = Flatten(item_embedding)
        
        # Merge
        # concat = concatenate()([user_embedding, item_embedding])
        concatenated = concatenate([user_embedding, item_embedding])
        dropout = Dropout(0.2)(concatenated)
        
        # layer 1
        layer1= Dense(64, activation='relu', name='layer1')(dropout)
        dropout1 = Dropout(0.2, name='dropout1')(layer1)
        batch_norm1 = BatchNormalization(name='batch_norm1')(dropout1)
        
        # layer 2
        layer2 = Dense(32, activation='relu',name='layer2')(batch_norm1)
        dropout2 = Dropout(0.2, name='dropout2')(layer2)
        batch_norm2 = BatchNormalization(name='batch_norm2')(dropout2)
        
        # layer 3
        layer3 = Dense(16, activation='relu', name='layer3')(batch_norm2)
        
        # layer 4
        layer4 = Dense(8, activation='relu', name='layer4')(layer3)
        
        # output
        output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(layer4)
        #lecun_uniform : [-limit, limit] 범위내 균등분포에 따라 샘플 생성
        
        # Model
        self.model = Model([user,item], output_layer)
        self.model.compile(optimizer='adam', loss='binary_crossentropy')
        
        def get_model(self):
            model = self.model
            return model

## GMF

In [ ]:

class GMF():
    def __init__(self, user_num, item_num):
        
        LATENT_SIZE = 8
        
        # user embedding
        user = Input((1,))
        user_embedding = Embedding(user_num, LATENT_SIZE, input_length = user.shape[1])(user)
        user_embedding = Flatten(user_embedding)
        
        # item embedding
        item = Input((1,))
        item_embedding = Embedding(item_num, LATENT_SIZE, input_length = item.shape[1])(item)
        item_embedding = Flatten(item_embedding)
        
        # Merge
        multi = Multiply([user_embedding, item_embedding])
        
        # output
        output_layer = Dense(1, activation='sigmoid', kernel_initializer = 'lecun_uniform', name = 'output_layer')(multi)

        # Model
        self.model = Model([user,item], output_layer)
        self.model.compile(optimizer='adam', loss= 'binary_crossentropy')
        
    def get_model(self):
        model = self.model
        return model

## NeuMF

In [7]:
class NeuMF():
    def __init__(self, user_num, item_num):
        LATENT_SIZE=8
        
        # input
        user = Input(1,)
        item = Input(1,)
        
        # MLP user embedding
        mlp_user_embedding = Embedding(user_num, 32, input_length = user.shape[1])(user)
        mlp_user_embedding = Flatten(mlp_user_embedding)
        
        # MLP item embedding
        mlp_item_embedding = Embedding(item_num, 32, input_length = item.shape[1])(item)
        mlp_item_embedding = Flatten(mlp_item_embedding)
        
        # GMF user embedding
        gmf_user_embedding = Embedding(user_num, LATENT_SIZE, input_length = user.shape[1])(user)
        gmf_user_embedding = Flatten(gmf_user_embedding)
        
        # GMF item embedding
        gmf_item_embedding = Embedding(item_num, LATENT_SIZE, input_length = item.shape[1])(item)
        gmf_item_embedding = Flatten(gmf_item_embedding)
        
        # MLP Merge
        mlp_concatenated = concatenate([mlp_user_embedding, mlp_item_embedding])
        mlp_dropout = Dropout(0.2)(mlp_concatenated)
        
        # GMF Merge
        gmf_multiply = Multiply([gmf_user_embedding, gmf_item_embedding])
        
        # MLP layer1
        mlp_layer1 = Dense(64, activation='relu', name='mlp_layer1')(mlp_dropout)
        mlp_dropout1 = Dropout(0.2)(mlp_layer1)
        mlp_batch_norm1 = BatchNormalization()(mlp_dropout1)
        
        # MLP layer2
        mlp_layer2 = Dense(32, activation='relu', name= 'mlp_layer2')(mlp_batch_norm1)
        mlp_dropout2 = Dropout(0.2)(mlp_layer2)
        mlp_batch_norm2 = BatchNormalization()(mlp_dropout2)
        
        # MLP layer3
        mlp_layer3 = Dense(16, activation='relu', name='mlp_layer3')(mlp_batch_norm2)
        
        # MLP layer4
        mlp_layer4 = Dense(8, activation='relu', name='mlp_layer4')(mlp_layer3)
        
        # MLP + GMF
        final_concatenated = concatenate([mlp_layer4, gmf_multiply])
        
        # output layer
        output_layer = Dense(1, activation='sigmoid', kernel_initializer = 'lecun_uniform', name=output_layer)(final_concatenated)
        
        # model
        self.model = Model([user,item], output_layer)
        self.model.compile(optimizer='adam', loss='binary_crossentropy')
        
    def get_model(self):
        model = self.model
        return model
        
        
        
        

# 평가 지표

In [ ]:
import numpy as np
import heapq

class Metrix:
    def __init__(self):
        pass
    
    def get_hits(self, k_ranked, holdout):
        for item in k_ranked:
            if item == holdout:
                return 1
        return 0
    
    def eval_rating(self, idx, test_ratings, test_negatives, K, model):
        # holdout(df_Test의 item)이 k순위 내에 있는지 평가하는 함수
        items = test_negatives[idx]
        user_idx = test_ratings[idx][0]
        holdout = test_ratings[idx][1]
        items.append(holdout)
        
        # prediction
        predict_user = np.full(len(items), user_idx, dtype='int32').reshape(-1,1)
        np_items = np.array(items).reshape(-1,1)
        
        predictions = model.predict([predict_user, np_items])
        predictions = predictions.flatten().tolist()
        item_to_pre_score = {item:pre for item, pre in zip(items, predictions)}
        
        # 점수가 높은 상위 k개 아이템 리스트
        k_ranked = heapq.nlargest(K,item_to_pre_score, key=item_to_pre_score.get)
        
        # holdout이 상위 K순위에 포함되는지
        # {1: 포함, 0: 포함x}
        hits = self.get_hits(k_ranked, holdout)
        
        return hits
    
    def evaluate_top_k(self, df_neg, df_test, model, K=10):
        # Top- K metric을 사용해 모델 평가하는 함수
        hits = []
        test_u = df_test['user_id'].values.tolist()
        test_i = df_test['item_id'].values.tolist()
        
        test_ratings = list(zip(test_u, test_i))
        df_neg = df_neg.drop(df_neg.columns[0], axis=1)
        test_negatives = df_neg.values.tolist()
        
        # user 샘플링
        sample_idx_lst = np.random.choice(len(test_ratings), int(len(test_ratings)))
        for user_idx in sample_idx_lst:
            
            hitrate = self.eval_rating(user_idx, test_ratings, test_negatives, K, model)
            hits.append(hitrate)
            
        return hits
        